In [1]:
from shogun import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets

In [2]:
df = pd.read_csv('../data/nyc-sales.csv')

In [3]:
for i in range(len(df)):
    if(df.loc[i,'SALE PRICE'] == ' -  '):
        df.at[i,'SALE PRICE'] = np.nan
        
# test is the data with 'SALE PRICE' == NaN
df_test = df[df['SALE PRICE'].isna()]

#train is the cleaned data
df_train = df[df['SALE PRICE'].notna()]


In [4]:
# training set
col_x = [col for col in df.columns
      if col != 'SALE PRICE']
X_train = df_train[col_x]

In [5]:
# target variable
target = ['SALE PRICE']
y_train = df_train[target]


In [6]:
# the string value that needed to be changed
#     if(val==[' -  ']):

# Test sets
y_test = df_test[target]
X_test = df_test[col_x]

In [7]:
# imputer not required
# myimputer = SimpleImputer(strategy='median')  

encoder = LabelEncoder()

s = (X_train.dtypes == 'object')

cat_vars = list(s[s].index)

# print(cat_vars)

In [8]:
# drop the unwanted columns
X_new = X_train.drop(columns=['Unnamed: 0','NEIGHBORHOOD','ADDRESS','EASE-MENT','BUILDING CLASS CATEGORY','SALE DATE','APARTMENT NUMBER'])

# X_new.head()

In [9]:
s = (X_new.dtypes == 'object')
cat_varsx = list(s[s].index)
# print(cat_varsx)

In [10]:
#  Labeling properly the columns with object data type
le = LabelEncoder()

label_X = X_new.copy()
for col in cat_varsx:
    label_X[col] = le.fit_transform(X_new[col])
    
# label_X contains the cleaned up training set


In [11]:
#Dropping some data to account for Memory error, see issue #2
label_X = label_X[:10000]
y_train = y_train[:10000]


In [12]:
# contains the error

from time import time
from sklearn.kernel_ridge import KernelRidge

t0 = time()
skr = KernelRidge(kernel='rbf', gamma=0.1)
skr.fit(label_X, y_train)
tsk1 = time() - t0
print(tsk1)

22.480963706970215


In [13]:
#Convert features and labels suitable for shogun

label_X= label_X.to_numpy().astype(float)
y_train = y_train.to_numpy().astype(float)
y_train = y_train.reshape(y_train.shape[0],) #reshape to (num_samples,)

In [14]:
from shogun import *


train_features = RealFeatures(label_X.T)
labels_train = RegressionLabels(y_train)


width = 0.1
kernel = GaussianKernel(train_features, train_features, width)
tau = 1

t0=time()
krr = KernelRidgeRegression(tau, kernel, labels_train)
krr.train()
tsg1=time()-t0

print(tsg1) 

65.07133722305298


In [15]:
#Final remarks
print('----------------------------------------')
print("############ Final remarks  ############")
print('----------------------------------------')
print("\n")
print("Sklearn training time: " + str(tsk1))
print("Shogun training time: " + str(tsg1))
print("Performance gap: " + str(tsg1 - tsk1))
print("\n")
print('----------------------------------------')


----------------------------------------
############ Final remarks  ############
----------------------------------------


Sklearn training time: 22.480963706970215
Shogun training time: 65.07133722305298
Performance gap: 42.590373516082764


----------------------------------------
